In [26]:
import os

import nibabel as nib
import numpy as np
import pandas as pd
from PIL import Image
from skimage import io
from tqdm import tqdm

In [27]:
# Get filenames and folder
def get_filenames(folder):
    filenames = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".nii.gz"):
                filenames.append(os.path.join(root, file))

    return filenames

In [28]:
# iterate over every image, load it, and save it as a png

image_files = get_filenames("data")
for file in tqdm(image_files):
    image = nib.load(file)
    image = image.get_fdata()
    image = image.astype(np.uint8)
    image = Image.fromarray(image)
    image.save(file.replace(".nii.gz", ".png"))
    os.remove(file)

100%|██████████| 916/916 [13:30<00:00,  1.13it/s]


Create the final datasets by loading the target, and saving only the layer containing the proximal femur

In [29]:
def get_targets(folder):
    filenames = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".png") and "targets" in root:
                filenames.append(os.path.join(root, file))
    return filenames

In [31]:
# Get all target filenames

target_files = get_targets("data")
for file in tqdm(target_files):
    image = io.imread(file)
    image = (image == 1).astype(np.uint8) * 255
    image = Image.fromarray(image)
    image.save(file)

100%|██████████| 458/458 [00:37<00:00, 12.12it/s]


In [32]:
# Check if the images are correct
files = get_targets("data")

data = []
for file in tqdm(files):
    image = io.imread(file)
    this_data = {"image":  file,
                 "max":    np.max(image),
                 "min":    np.min(image),
                 "unique": len(np.unique(image))
                 }
    data.append(this_data)
df = pd.DataFrame(data)

100%|██████████| 458/458 [00:55<00:00,  8.27it/s]
